In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36r'}
baseurl = "https://www.clickbd.com";

In [4]:
r=requests.get(baseurl + '/h_browse_by_category')

In [5]:
soup = BeautifulSoup(r.content, 'lxml')

In [6]:
soup

<html><body><div class="panes">
<div class="detail">
<h4>Mobiles</h4>
<ul class="parent">
<li><a href="/search?category=cases">Cases</a></li>
<li><a href="/search?category=mobile-accessories">Mobile Accessories</a></li>
<li><a href="/search?category=mobile-internet">Mobile Internet</a></li>
<li><a href="/search?category=mobile-phones">Mobile Phones</a></li>
<li><a href="/search?category=sim-cards">SIM Cards</a></li>
<li><a href="/search?category=wearable-technology">Wearable Technology</a></li>
</ul>
</div>
</div>
<div class="panes">
<div class="detail">
<h4>Computers</h4>
<ul class="parent">
<li><a href="/search?category=computer-accessories">Computer Accessories</a></li>
<li><a href="/search?category=computer-parts">Computer Parts</a></li>
<li><a href="/search?category=desktops">Desktops</a></li>
<li><a href="/search?category=drives-and-storage">Drives &amp; Storage</a></li>
<li><a href="/search?category=ebook-readers">eBook Readers</a></li>
<li><a href="/search?category=laptops">Lap

In [7]:
categories = soup.findAll('div', {'class': 'panes'})

In [8]:
print(categories)

[<div class="panes">
<div class="detail">
<h4>Mobiles</h4>
<ul class="parent">
<li><a href="/search?category=cases">Cases</a></li>
<li><a href="/search?category=mobile-accessories">Mobile Accessories</a></li>
<li><a href="/search?category=mobile-internet">Mobile Internet</a></li>
<li><a href="/search?category=mobile-phones">Mobile Phones</a></li>
<li><a href="/search?category=sim-cards">SIM Cards</a></li>
<li><a href="/search?category=wearable-technology">Wearable Technology</a></li>
</ul>
</div>
</div>, <div class="panes">
<div class="detail">
<h4>Computers</h4>
<ul class="parent">
<li><a href="/search?category=computer-accessories">Computer Accessories</a></li>
<li><a href="/search?category=computer-parts">Computer Parts</a></li>
<li><a href="/search?category=desktops">Desktops</a></li>
<li><a href="/search?category=drives-and-storage">Drives &amp; Storage</a></li>
<li><a href="/search?category=ebook-readers">eBook Readers</a></li>
<li><a href="/search?category=laptops">Laptops</a></

In [ ]:
product_list = []
empty_file = True
for category in categories:
    div = category.find('div', {'class': 'detail'})
    category_title = div.find('h4').text #attr
    subcategories = div.find('ul').findAll('li')
    for subcategory in subcategories:
        a = subcategory.find('a', href=True)
        subcategory_title = a.text #attr
        subcategory_link = a.attrs['href'] #attr
        r=requests.get(baseurl + subcategory_link, headers=headers)
        soup = BeautifulSoup(r.content, 'lxml')
        next_enabled = True;
        while next_enabled:
            products = soup.find('div', {'id': 'view'}).findAll('div', {'class': 'sh'})
            product_list = []
            for product in products:
                try:
                    a = product.find('a', href=True)
                    left_div = product.find('div', {'class': 'lt'})
                    product_link = a.attrs['href'] #attr
                    product_image = 'https:' + a.find('img').attrs['src'] #attr
                    product_title = left_div.find('h3').find('a').text #attr
                    i = left_div.find('h3').find('i')
                    product_label = i.text if i != None else None #attr
                    i = product.select('div.lt > i')[0]
                    product_date = i.text if i != None else None #attr
                    right_div = product.find('div', {'class': 'rt'})
                    product_price = right_div.find('b').find('b').text #attr
                    s = right_div.find('div').find('s')
                    strong = right_div.find('div').find('strong')
                    product_price_before = s.text if s != None and strong != None else None #attr
                    product_price_off = strong.text if s != None and strong != None else None #attr
                    product_right_label = right_div.find('i').text #attr
                    product_buynow_label = 1 if right_div.find('span', {'class': 'i-buy-it-now'}) else 0 #attr
                    p_r=requests.get(baseurl + product_link, headers=headers)
                    p_soup = BeautifulSoup(p_r.content, 'lxml')
                    product_description_lines = p_soup.find('div', {'class': 'description'}).findAll('p')
                    product_description = '';
                    for product_description_line in product_description_lines:
                        product_description += '\n' + product_description_line.text if product_description_line else None
                    info_items = p_soup.find('div', {'class': 'info'}).find('div', {'class': 'delivery_info'}).findAll('div', {'class': 'row'})
                    product = {
                        'category_title': category_title,
                        'subcategory_title': subcategory_title,
                        'subcategory_link': subcategory_link,
                        'link': product_link,
                        'image': product_image,
                        'title': product_title,
                        'label': product_label,
                        'right_label': product_right_label,
                        'date': product_date,
                        'price': product_price,
                        'price_before': product_price_before,
                        'price_off': product_price_off,
                        'buynow_label': product_buynow_label,
                        'description': product_description,
                    }
                    for item in info_items:
                        product_attr = item.find('div', {'class': 'col-xs-3'})
                        product_value = item.find('div', {'class': 'col-xs-9'})
                        if product_attr != None and product_attr.text != None and product_value != None:
                            product[product_attr.text[0:len(product_attr.text)-1]] = product_value.text if product_value.text else str(product_value)
                    product_list.append(product)
                except AttributeError:
                    pass
            df = pd.DataFrame(product_list)
            df.to_csv('clickbd_2021101514.csv', mode='a', header=empty_file)
            empty_file = False
            pages = soup.find('div', {'class': 'pagn'}).find('ul', {'class': 'pagination'}).findAll('li')
            if len(pages) > 0:
                next_button = pages[-1]
                link = next_button.find('a', href=True)
                if link != None:
                    print(link.attrs['href'])
                    r=requests.get(baseurl + link.attrs['href'], headers=headers)
                    soup = BeautifulSoup(r.content, 'lxml')
                else:
                    print(subcategory_title)
                    time.sleep(5)
                    next_enabled = False
            else:
                print(subcategory_title)
                time.sleep(5)
                next_enabled = False

/search?category=cases&page=30
Cases
/search?category=mobile-accessories&page=30
/search?category=mobile-accessories&page=60
/search?category=mobile-accessories&page=90
/search?category=mobile-accessories&page=120
/search?category=mobile-accessories&page=150
/search?category=mobile-accessories&page=180
/search?category=mobile-accessories&page=210
/search?category=mobile-accessories&page=240
/search?category=mobile-accessories&page=270
/search?category=mobile-accessories&page=300
/search?category=mobile-accessories&page=330
/search?category=mobile-accessories&page=360
/search?category=mobile-accessories&page=390
/search?category=mobile-accessories&page=420
/search?category=mobile-accessories&page=450
/search?category=mobile-accessories&page=480
/search?category=mobile-accessories&page=510
/search?category=mobile-accessories&page=540
/search?category=mobile-accessories&page=570
/search?category=mobile-accessories&page=600
/search?category=mobile-accessories&page=630
/search?category=mobi

In [13]:
!cp clickbd_2021101514.cp "drive/My Drive/"